# **Estruturas criptograficas: TP2 problema 1**

In [197]:
import hashlib, os 
from sage.all import *

In [198]:
class EdwardsPoint(object):
    
    def __init__(self, params, x, y):
        self.params = params
        self.curve = params['EC']
        self.x = x
        self.y = y
        self.w = x*y
    
    
    def copy(self):
        return EdwardsPoint(self.params, self.x, self.y)
    
    def zero(self):
        return EdwardsPoint(self.params, 0, 1)
       
    def duplica(self):
        a = self.params['a']
        d = self.params['d']
        delta = d*(self.w)**2
        self.x, self.y = (2*self.w)/(1+delta) , (self.y**2 - a*self.x**2)/(1 - delta)
        self.w = self.x*self.y
           
    def __add__(self, other):
        a = self.params['a']
        d = self.params['d']
        delta = d*self.w*other.w
        self.x, self.y  = (self.x*other.y + self.y*other.x)/(1+delta), (self.y*other.y - a*self.x*other.x)/(1-delta)
        self.w = self.x*self.y
        return self
    
    def __mul__(self, n):
        m = Mod(n, self.params['L']).lift().digits(2)
        Q = self.copy()
        A = self.zero()
        for b in m:
            if b == 1:
                A + Q
            Q.duplica()
        return A
    
    def __eq__(self,other):
        return self.x == other.x and self.y == other.y

In [199]:
class EdDSA:
    
    def __init__(self):
        self.b = 256
        self.p = 2^255 - 19
        self.K = GF(self.p)
        self.x = self.K(15112221349535400772501151409588531511454012693041857206046113283949847762202)
        self.y = self.K(46316835694926478169428394003475163141307993866256225615783033603165251855960)
        self.c = 3
        self.n = 254
        self.d = 37095705934669439343138083508754565189542113879843219016388785533085940283555
        self.a = self.K(-1)
        self.L = 2^252 + 27742317777372353535851937790883648493
        
        # Calular Edwards Curve     
        self.A =  2*(self.a + self.d)/(self.a - self.d)
        self.B =  4/(self.a - self.d)
    
        self.alfa = self.A/(3*self.B)
        self.beta = self.B

        self.a4 =  self.beta**(-2) - 3*self.alfa**2
        self.a6 =  -self.alfa**3 - self.a4*self.alfa
        
        self.EC = EllipticCurve(self.K,[self.a4,self.a6])
         
        self.params = {
            'd': self.d,
            'a': self.a,
            'L': self.L,
            'EC': self.EC
        }
         
        self.B = EdwardsPoint(self.params, self.x, self.y)       
            
    def H(self, k):
        return hashlib.sha512(k).digest()    
        
    def frombytes(self,x,b):
        rv = int.from_bytes(x, byteorder="little") % (2**(b-1))
        return self.K(rv) if rv < self.p else None
    
    def solve_x2(self,y):
        return ((y*y-self.K(1))/(self.d*y*y+self.K(1)))
    
    def sqrt(self, x):
        y = pow(x,(self.p+3)//8,self.p)
        
        if (y * y) % self.p == x % self.p: return y
        else:
            z = pow(2,(self.p - 1)//4, self.p)
            return (y * z) % self.p
    
    def decoding(self,s):
        s = bytearray(s)
        if len(s)!=self.b//8: 
            return (None,None)
        
        xs=s[(self.b-1)//8]>>((self.b-1)&7)
        y = self.frombytes(s,self.b)
        
        if y is None: 
            return (None,None)
        
        x = self.solve_x2(y)
        x = self.sqrt(x)
        
        if x is None or (x == 0 and xs != x % 2):
            return (None,None)
        
        if mod(x, 2) != xs: 
            x=-x
        
        return (x,y)
    
    def encoding(self, x, y):
        s = bytearray(int(y).to_bytes(self.b//8,byteorder="little"))
        
        if mod(x, 2) != 0: 
            s[(self.b-1)//8]|=1 << (self.b-1) % 8
            
        return bytes(s)
            
    def keygen(self):
        private_key = os.urandom(self.b / 8)
        
        pk = self.H(private_key)
        pk = pk[:32]
        
        bits = int.from_bytes(pk)
    
        bits = [int(digit) for digit in list(ZZ(bits).binary())]
        
        extra = self.b - len(bits)
        for i in range(extra):
            bits.insert(0, 0)
        
        bits[0] = bits[1] = bits[2] = 0
        bits[self.b-2] = 1
        bits[self.b-1] = 0
        
        bits = "".join(map(str, bits))

        s = int(bits[::-1], 2)
        
        Q = self.B * s
        Q_enc = self.encoding(Q.x, Q.y)
        
        Q_int = int.from_bytes(Q_enc, 'little')
        public_key = int(Q_int).to_bytes(self.b/8, 'little')
        
        return private_key, public_key
                
    #TODO: fazer o PH caso a mensagem seja muito grande
    def sign(self, private_key, public_key, message):
        
        d = self.H(private_key)
        pk = d[32:]
        
        M = message.encode()
        
        r = self.H(pk + M)
        r = int.from_bytes(r, 'little')
        
        P = self.B * r
        R = self.encoding(P.x, P.y)
        
        hd = d[:32]
        bits = int.from_bytes(hd)
    
        bits = [int(digit) for digit in list(ZZ(bits).binary())]
        
        extra = self.b - len(bits)
        for i in range(extra):
            bits.insert(0, 0)
        
        bits[0] = bits[1] = bits[2] = 0
        bits[self.b-2] = 1
        bits[self.b-1] = 0
        
        bits = "".join(map(str, bits))

        s = int(bits[::-1], 2)
        #--------------------------------------
        
        digest = self.H(R + public_key + M)
        digest = int.from_bytes(digest, 'little')
        
        S = mod(r + digest * s, self.L)
        
        return R + int(S).to_bytes(self.b/8, 'little')
    
    def verify(self, M, public_key, signature):
        
        R = signature[:32]
        t = signature[32:]
        
        t = int.from_bytes(t, 'little')
        
        if t < 0 or t >= self.L:
            print("Signature Invalid!")
            return
        
        digest = self.H(R + public_key + M.encode())
        u = int.from_bytes(digest, 'little')
        
        R = self.decoding(R)
        R = EdwardsPoint(self.params, R[0], R[1])
        
        Q = self.decoding(public_key)
        Q = EdwardsPoint(self.params, Q[0], Q[1])
        
        b1 = self.B * t
        q1 = Q * u
        r1 = R + q1
        
        valid = b1 == r1 
        
        if valid is True:
            print("Valid Signature!")
        else:       
            print("Invalid Signature!")
       
   

In [200]:
edsa = EdDSA()
sk, pk = edsa.keygen()
print((sk, pk))

print('-----------')

signature = edsa.sign(sk, pk, "SageMath ser grande merda")

edsa.verify("SageMath ser grande merda", pk, signature)

(b'\x84\xca\xda\xc3\xcfK2\xe1S\x88\xb3\x97\xae\xbf\nk2\xc3\x15\xe4 \xfe\xa9^\xa7(\xfbD\xcc9\x86\xd8', b'\x84}\x96\x1e\xebC\xd4\x82gd\xb5\xe0\x8f\xe7\x12F\x10\xbc\xd0E\x0f\xd2\xf9 cD\x0b\x13\x8e;\xf5w')
-----------
Valid Signature!
